<a href="https://colab.research.google.com/github/csalnav2/PhotonMechanic/blob/main/ComputationalGraphProfiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok --quiet # Install the pyngrok module
from pyngrok import ngrok

In [ ]:
!ngrok config add-authtoken 2tedizGJDmEJMDtfhCR6ftx8EyX_2RaA7t8nPn17SU7jiyK1H

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!kill -9 $(lsof -t -i:8000) 2>/dev/null || echo "No existing process"


No existing process


In [ ]:
!pip install --quiet flask transformers torch networkx matplotlib scikit-learn safetensors pyvis soundfile psutil GPUtil
# no dynamic quant error on GPU for int8 => fallback to CPU

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.7 MB/s eta 0:00:00


In [ ]:
!pkill -f ngrok

In [1]:
# =============================================
# Single Cell or local Python script
#   - Kills old processes on port=8000
#   - Installs dependencies
#   - Writes interpret_flask.py
#   - Runs Flask in background
#   - Polls for readiness or times out
#   - Then starts ngrok
# =============================================

import os, time, requests, random
from subprocess import Popen

def kill_port_8000():
    try:
        os.system("pkill -f ngrok 2>/dev/null || echo 'No existing ngrok process.'")
        os.system("kill -9 $(lsof -t -i:8000) 2>/dev/null || echo 'No existing process.'")
    except:
        pass

print("🔴 Killing anything on port=8000 + old ngrok sessions.")
kill_port_8000()

print("📦 Installing dependencies (quietly)...")
try:
    os.system("pip install --quiet flask transformers torch networkx matplotlib pyvis psutil requests pyngrok scipy GPUtil plotly")
except:
    pass

FULL_CODE = r'''
###############################################################################
# interpret_flask.py
#
# BFS-based interpretability with:
#  - Memory/time fallback for unknown shapes
#  - Per-node stable random factor => unique FLOPs
#  - Single scroll
#  - 3D Q-Space
#  - Data export endpoints
#  - REMOVES conversation logs
#  - ADDS /analysis_3d_scatter => memory vs. time vs. FLOPs 3D Plotly
#    with BFS distance as color
###############################################################################

import os
import time
import random
import threading
import traceback
import math
import torch
import torch.nn as nn
import torch.fx as fx
import networkx as nx
import psutil
import GPUtil
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from io import BytesIO
from flask import Flask, request, send_file, jsonify, Response

from transformers import AutoModel, AutoTokenizer
from pyvis.network import Network
from scipy.interpolate import griddata
import csv
import json
import requests

app = Flask(__name__, static_folder="static")
if not os.path.exists("static"):
    os.makedirs("static")

###############################################################################
# GLOBALS
###############################################################################

RL_CHOICE = "No RL"
HYP_GRAPH = None
COMP_GRAPH = None
OLD_HYP_GRAPH = None
OLD_COMP_GRAPH = None

MODEL_CACHE = {}

COMP_SHAPE_INFO = {}  # { "COMP_nodeName": (in_shape, out_shape) }

Q_TABLE = {s:[random.uniform(-1,1) for _ in range(4)] for s in range(10)}

GPU_MEMORY_SCALE = {
    "T4":1.0,
    "A100":0.5,
    "V100":0.7,
    "TPU":0.4
}

COMPLEXITY_OPTS = ["Quadratic (n^2)", "Logarithmic (n log n)", "Linear (n)"]

AVAILABLE_MODELS = {
    "BERT (base)":"bert-base-uncased",
    "GPT-2":"gpt2",
    "DistilBERT":"distilbert-base-uncased"
}
AVAILABLE_GPUS = ["T4","A100","V100","TPU"]
QUANT_METHODS = ["None (FP32)","FP16","INT8"]
REASONING_METHODS = [
    "No reasoning",
    "Monte Carlo Tree Search (MCTS)",
    "Chain of Thought (CoT)",
    "Meta-CoT",
    "Neural-Symbolic"
]
RL_ALGORITHMS = [
    "No RL",
    "Q-Learning",
    "Deep Q-Network (DQN)",
    "Policy Gradient",
    "Actor-Critic",
    "PPO"
]

FLOP_RANDOM_FACTORS = {}  # stable random factor for each node name => unique FLOPs

###############################################################################
# BFS
###############################################################################

def compute_bfs_levels(nx_graph):
    if nx_graph.number_of_nodes()==0:
        return
    for n in nx_graph.nodes:
        nx_graph.nodes[n]["BFS_distance"]=0

    root_candidates = [n for n in nx_graph.nodes if "input" in n.lower()]
    if not root_candidates:
        in_degs = dict(nx_graph.in_degree())
        zero_in = [nd for nd,deg in in_degs.items() if deg==0]
        if zero_in:
            root_candidates= zero_in
    if not root_candidates:
        root_candidates= [list(nx_graph.nodes)[0]]

    visited= set()
    queue= []
    for r in root_candidates:
        queue.append((r,0))
        nx_graph.nodes[r]["BFS_distance"]=0
        visited.add(r)

    while queue:
        current, dist= queue.pop(0)
        for neighbor in nx_graph.neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                nx_graph.nodes[neighbor]["BFS_distance"]= dist+1
                queue.append((neighbor, dist+1))

###############################################################################
# TIME + FLOPs => stable random factor
###############################################################################

def stable_flop_factor(node_str):
    """ Return a stable random factor for each node_str => unique FLOPs/time. """
    global FLOP_RANDOM_FACTORS
    if node_str not in FLOP_RANDOM_FACTORS:
        FLOP_RANDOM_FACTORS[node_str] = random.uniform(0.5,2.0)
    return FLOP_RANDOM_FACTORS[node_str]

def interpret_time_cost(node_str, attn_comp, lin_comp, def_comp, seq_len):
    low = node_str.lower()
    if "attention" in low or "matmul" in low:
        chosen = attn_comp
    elif "linear" in low:
        chosen = lin_comp
    else:
        chosen = def_comp

    base = 0
    if chosen.startswith("Quadratic"):
        base = seq_len**2
    elif chosen.startswith("Logarithmic"):
        base = int(seq_len*math.log2(seq_len+1))
    else:
        base = seq_len

    factor = stable_flop_factor(node_str)  # stable factor
    return base*factor

def compute_node_flops(node_str, attn_comp, lin_comp, def_comp, seq_len):
    return interpret_time_cost(node_str, attn_comp, lin_comp, def_comp, seq_len)

###############################################################################
# Torch FX shape tracing
###############################################################################

def trace_model_fx(model, tokenizer):
    device = next(model.parameters()).device
    wrap = ModelWrapper(model, tokenizer).to(device)
    enc = tokenizer("Hello world!", return_tensors="pt").to(device)

    shape_info = {}
    try:
        traced = fx.symbolic_trace(
            wrap,
            concrete_args={
                "input_ids": enc["input_ids"],
                "attention_mask": enc.get("attention_mask", None)
            }
        )
        from torch.fx.passes.shape_prop import ShapeProp
        sp = ShapeProp(traced)
        sp.run(enc)

        for node in traced.graph.nodes:
            node_name = str(node)
            in_shape  = "?"
            out_shape = "?"
            try:
                if "tensor_meta" in node.meta:
                    out_shape = tuple(node.meta["tensor_meta"].shape)
            except:
                pass
            shape_info[node_name] = (in_shape, out_shape)
    except:
        pass
    return shape_info

def estimate_gpu_memory(node_name, shape_tuple):
    if not isinstance(shape_tuple, tuple):
        return "Unknown"
    numel = 1
    for dim in shape_tuple:
        numel*= dim
    mem_kb = (numel*4)/1024
    return f"{mem_kb:.1f}"

def interpret_operation(node_name):
    low = node_name.lower()
    if "matmul" in low:
        return "MatrixMultiplication"
    elif "transpose" in low:
        return "Transpose"
    elif "softmax" in low:
        return "Softmax"
    elif "norm" in low:
        return "LayerNorm"
    elif "activation" in low or "gelu" in low or "relu" in low:
        return "Activation"
    elif "linear" in low or "dense" in low:
        return "Linear"
    elif "dropout" in low:
        return "Dropout"
    elif "pooler" in low or "pool" in low:
        return "Pooling"
    elif "attention" in low:
        return "Attention block"
    return "Generic op"

###############################################################################
# MODEL UTILS
###############################################################################

def load_model(hf_id, quant="None (FP32)"):
    import torch.nn as nn
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if quant=="INT8" and device=="cuda":
        device="cpu"

    cache_key = f"{hf_id}_{quant}_{device}"
    if cache_key in MODEL_CACHE:
        return MODEL_CACHE[cache_key]["model"], MODEL_CACHE[cache_key]["tokenizer"]

    model = AutoModel.from_pretrained(hf_id)
    tokenizer = AutoTokenizer.from_pretrained(hf_id)
    if quant=="FP16":
        model = model.half()
    elif quant=="INT8":
        model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

    model.eval()
    if quant!="INT8":
        model.to(device)

    MODEL_CACHE[cache_key] = {"model": model, "tokenizer": tokenizer}
    return model, tokenizer

def toy_finetune(model):
    device = next(model.parameters()).device
    model.train()
    x = torch.randint(0,1000,(2,8), device=device)
    m = torch.ones_like(x)
    out = model(input_ids=x, attention_mask=m)
    if isinstance(out, tuple) and out and isinstance(out[0], torch.Tensor):
        loss = out[0].sum()
    elif hasattr(out, "last_hidden_state"):
        loss = out.last_hidden_state.sum()
    else:
        loss = torch.tensor(0.0, device=device)
    optim = torch.optim.AdamW(model.parameters(), lr=1e-5)
    optim.zero_grad()
    loss.backward()
    optim.step()
    model.eval()

class ModelWrapper(nn.Module):
    def __init__(self, submodel, tokenizer):
        super().__init__()
        self.submodel = submodel
        self.tokenizer = tokenizer
    def forward(self, input_ids, attention_mask=None):
        return self.submodel(input_ids=input_ids, attention_mask=attention_mask)

###############################################################################
# RL Q-Table
###############################################################################

def update_q_table(state, action, reward):
    old_val= Q_TABLE[state][action]
    new_val= old_val + 0.1*(reward - old_val)
    Q_TABLE[state][action]= new_val

###############################################################################
# GRAPH EXTRACTION
###############################################################################

def fallback_comp_graph_for_gpt2(model, reasoning, gpu_sel):
    import networkx as nx
    g= nx.Graph()
    for name,module in model.named_modules():
        node_name= "COMP_"+ name
        g.add_node(node_name,
                   BFS_distance=0,
                   mem=0,target_mem=0,
                   timecost=0,target_time=0,
                   flops=0)
        if "." in name:
            parent= "COMP_"+ name.rsplit(".",1)[0]
            child= "COMP_"+ name
            g.add_edge(parent, child)
    return g

def extract_comp_graph(model, tokenizer, gpu_sel, quant, reasoning):
    import networkx as nx
    import torch.fx as fx
    try:
        device= next(model.parameters()).device
        wrap= ModelWrapper(model, tokenizer).to(device)
        enc= tokenizer("Hello world!",return_tensors="pt").to(device)

        traced= fx.symbolic_trace(
            wrap,
            concrete_args={
                "input_ids": enc["input_ids"],
                "attention_mask": enc.get("attention_mask", None)
            }
        )
        g= nx.Graph()
        for node in traced.graph.nodes:
            nm= "COMP_"+ str(node)
            g.add_node(nm,
                       BFS_distance=0,
                       mem=0,target_mem=0,
                       timecost=0,target_time=0,
                       flops=0)
        for node in traced.graph.nodes:
            for user in node.users:
                g.add_edge("COMP_"+ str(node), "COMP_"+ str(user))
        return g
    except:
        return fallback_comp_graph_for_gpt2(model, reasoning, gpu_sel)

def extract_hypergraph(model, quant, reasoning):
    import networkx as nx
    g= nx.Graph()
    for name,module in model.named_modules():
        node_name= "HYP_"+ name
        g.add_node(node_name)
        if "." in name:
            parent= "HYP_"+ name.rsplit(".",1)[0]
            child= "HYP_"+ name
            g.add_edge(parent,child)
    return g

def get_layer_subgraph(nx_graph, layer_num):
    import networkx as nx
    subg= nx.Graph()
    if nx_graph.number_of_nodes()==0:
        return subg
    if "BFS_distance" not in list(nx_graph.nodes(data=True))[0][1]:
        compute_bfs_levels(nx_graph)

    for n,d in nx_graph.nodes(data=True):
        if d.get("BFS_distance",None)== layer_num:
            subg.add_node(n,**d)
    for (u,v) in nx_graph.edges():
        if (u in subg.nodes) and (v in subg.nodes):
            subg.add_edge(u,v,**nx_graph[u][v])
    return subg

###############################################################################
# Q-Space route
###############################################################################

@app.route("/q_space")
def q_space_3d():
    import numpy as np
    from scipy.interpolate import griddata
    states= sorted(Q_TABLE.keys())
    xs,ys,zs= [],[],[]
    for si,s in enumerate(states):
        for ai,val in enumerate(Q_TABLE[s]):
            xs.append(float(si))
            ys.append(float(ai))
            zs.append(val)
    xs= np.array(xs)
    ys= np.array(ys)
    zs= np.array(zs)

    minZ,maxZ= zs.min(), zs.max()
    if abs(maxZ-minZ)<1e-9:
        z_scaled= np.full_like(zs,4.5)
    else:
        z_scaled= 9.0*(zs-minZ)/(maxZ-minZ)

    def polynomial_fit_r2(xs, ys, zs):
        X= np.column_stack((np.ones_like(xs),xs,ys, xs*xs, xs*ys, ys*ys))
        c,res,rank,s= np.linalg.lstsq(X,zs,rcond=None)
        z_pred= X@c
        ss_res= np.sum((zs-z_pred)**2)
        ss_tot= np.sum((zs- np.mean(zs))**2)
        if ss_tot<1e-12:
            return 1.0
        return 1-(ss_res/ss_tot)

    r2_value= polynomial_fit_r2(xs,ys,z_scaled)

    Xi= np.linspace(0,9,10)
    Yi= np.linspace(0,9,10)
    XiG,YiG= np.meshgrid(Xi,Yi)
    from scipy.interpolate import griddata
    Zi= griddata((xs,ys),z_scaled,(XiG,YiG), method="cubic")
    if np.isnan(Zi).any():
        Zi2= griddata((xs,ys),z_scaled,(XiG,YiG), method="nearest")
        Zi[np.isnan(Zi)] = Zi2[np.isnan(Zi)]

    import plotly.graph_objects as go
    fig= go.Figure(data=[go.Surface(z=Zi, x=Xi, y=Yi, colorscale="Viridis")])
    fig.update_traces(contours_z=dict(show=True,usecolormap=True,highlightcolor="limegreen",project_z=True))
    fig.update_layout(
        title="3D Q-Space",
        autosize=False,
        width=700,height=600,
        margin=dict(l=65,r=50,b=65,t=90),
        scene=dict(
            xaxis=dict(title="State", range=[0,9],autorange=False),
            yaxis=dict(title="Action", range=[0,9],autorange=False),
            zaxis=dict(title="Q-Value", range=[0,9],autorange=False),
            camera=dict(eye=dict(x=1.6,y=1.6,z=1.2)),
            aspectmode="cube"
        )
    )
    fig.update_layout(scene_dragmode="orbit")
    fig.write_html("static/qspace_3d.html")

    return f"""
<h2>3D Q-Space</h2>
<p>R²={r2_value:.4f}</p>
<iframe src="/static/qspace_3d.html" width="700" height="600"
  style="border:none;overflow:hidden;" scrolling="no"></iframe>
"""

###############################################################################
# analysis_3d_scatter => memory vs. time vs. FLOPs, color=BFS
###############################################################################

@app.route("/analysis_3d_scatter")
def analysis_3d_scatter():
    """
    3D scatter of memory (x), time (y), flops (z), BFS distance = color
    """
    global COMP_GRAPH
    if not COMP_GRAPH or COMP_GRAPH.number_of_nodes()==0:
        return "<h2>No CompGraph found. Please run first!</h2>"

    import numpy as np
    import plotly.graph_objects as go

    xs, ys, zs, colors, texts = [], [], [], [], []

    for n,d in COMP_GRAPH.nodes(data=True):
        BFS_d= d.get("BFS_distance",0)
        memv= d.get("mem",0)
        timv= d.get("timecost",0)
        flp = d.get("flops",0)
        xs.append(memv)     # x= memory
        ys.append(timv)     # y= time
        zs.append(flp)      # z= flops
        colors.append(BFS_d) # BFS distance => color scale
        texts.append(n)

    fig= go.Figure(data=[
        go.Scatter3d(
            x=xs, y=ys, z=zs,
            mode='markers',
            text=texts,
            marker=dict(
                size=5,
                color=colors,
                colorscale='Viridis',
                showscale=True
            )
        )
    ])

    fig.update_layout(
        title="Memory vs. Time vs. FLOPs (color=BFS distance)",
        scene=dict(
            xaxis=dict(title="Memory (KB)"),
            yaxis=dict(title="Time (ms)"),
            zaxis=dict(title="FLOPs"),
            camera=dict(eye=dict(x=1.3,y=1.3,z=1.0))
        ),
        width=700, height=600,
        margin=dict(l=65,r=50,b=65,t=90)
    )
    fig.update_layout(scene_dragmode="orbit")

    # write to static
    outpath="static/analysis_3d.html"
    fig.write_html(outpath)

    return f"""
<h2>3D Scatter: Memory vs. Time vs. FLOPs</h2>
<p>BFS distance = color scale. Drag to orbit!</p>
<iframe src="/static/analysis_3d.html" width="700" height="600"
        style="border:none;overflow:hidden;" scrolling="no">
</iframe>
"""

###############################################################################
# GLOSSARIES
###############################################################################

@app.route("/glossary_hyper")
def glossary_hyper():
    global HYP_GRAPH
    if not HYP_GRAPH or HYP_GRAPH.number_of_nodes()==0:
        return "<h2>No Hypergraph found</h2>"
    module_desc= {
        "attention":"Multi-head self-attention mechanism",
        "ffn":"Feedforward network",
        "layernorm":"LayerNorm for stable activations",
        "dropout":"Regularization to prevent overfitting",
        "embedding":"Embedding layer",
        "pooler":"Pooling for final representation"
    }
    html= "<table border='1' cellpadding='6' cellspacing='0'><tr><th>Node</th><th>Description</th></tr>"
    for n in sorted(HYP_GRAPH.nodes()):
        desc="General model component"
        low_n= n.lower()
        for key in module_desc:
            if key in low_n:
                desc= module_desc[key]
                break
        html+= f"<tr><td>{n}</td><td>{desc}</td></tr>"
    html+="</table>"
    return f"<h2>Hypergraph Glossary</h2>{html}"

@app.route("/glossary_comp")
def glossary_comp():
    global COMP_GRAPH, COMP_SHAPE_INFO
    if not COMP_GRAPH or COMP_GRAPH.number_of_nodes()==0:
        return "<h2>No CompGraph found</h2>"

    html= "<table border='1' cellpadding='6' cellspacing='0'><tr>"
    html+= "<th>Node</th><th>Operation</th><th>BFS dist</th>"
    html+= "<th>Input shape</th><th>Output shape</th><th>GPU Mem (KB)</th>"
    html+= "<th>Reason (if ? or Unknown)</th>"
    html+="</tr>"

    for n,d in sorted(COMP_GRAPH.nodes(data=True)):
        bfs_d= d.get("BFS_distance","?")
        op_type= interpret_operation(n)
        in_shape,out_shape= COMP_SHAPE_INFO.get(n, ("?","?"))
        mem_kb= estimate_gpu_memory(n,out_shape)
        reason=""
        if in_shape=="?" or out_shape=="?":
            reason+="Unknown shape => fallback random size. "
        if mem_kb=="Unknown":
            reason+="No shape => can't compute memory."

        html+= f"<tr><td>{n}</td><td>{op_type}</td><td>{bfs_d}</td>"
        html+= f"<td>{in_shape}</td><td>{out_shape}</td><td>{mem_kb}</td>"
        html+= f"<td>{reason}</td></tr>"
    html+="</table>"
    return f"<h2>CompGraph Glossary</h2>{html}"

###############################################################################
# EXPORTS
###############################################################################

def graph_to_json(nx_graph):
    data_out= {"nodes":[],"edges":[]}
    for n,d in nx_graph.nodes(data=True):
        item={"name":n}
        for k,v in d.items():
            item[k]=v
        data_out["nodes"].append(item)
    for s,t in nx_graph.edges():
        e_data= nx_graph[s][t]
        data_out["edges"].append({
            "source":s,
            "target":t,
            "color": e_data.get("color","gray")
        })
    return data_out

def graph_to_csv(nx_graph):
    import io
    output= io.StringIO()
    output.write("NODES\n")
    output.write("name,BFS_distance,mem,target_mem,timecost,target_time,color,flops\n")
    for n,d in nx_graph.nodes(data=True):
        bfs= d.get("BFS_distance","")
        m  = d.get("mem","")
        tm = d.get("target_mem","")
        tc = d.get("timecost","")
        ttc= d.get("target_time","")
        col= d.get("color","")
        flp= d.get("flops","")
        output.write(f"{n},{bfs},{m},{tm},{tc},{ttc},{col},{flp}\n")
    output.write("\nEDGES\n")
    output.write("source,target,color\n")
    for s,t in nx_graph.edges():
        e_data= nx_graph[s][t]
        c= e_data.get("color","gray")
        output.write(f"{s},{t},{c}\n")
    csv_str= output.getvalue()
    output.close()
    return csv_str

@app.route("/export_hypergraph")
def export_hypergraph_json():
    global HYP_GRAPH
    if not HYP_GRAPH:
        return jsonify({"error":"No hypergraph to export"})
    data_out= graph_to_json(HYP_GRAPH)
    return jsonify({"message":"Hypergraph exported","data":data_out})

@app.route("/export_hypergraph_csv")
def export_hypergraph_csv():
    global HYP_GRAPH
    if not HYP_GRAPH:
        return Response("No hypergraph to export",status=400)
    csv_str= graph_to_csv(HYP_GRAPH)
    return Response(csv_str,mimetype="text/csv",
                    headers={"Content-disposition":"attachment; filename=hypergraph_data.csv"})

@app.route("/export_comp_graph")
def export_comp_graph_json():
    global COMP_GRAPH
    if not COMP_GRAPH:
        return jsonify({"error":"No comp graph to export"})
    data_out= graph_to_json(COMP_GRAPH)
    return jsonify({"message":"CompGraph exported","data":data_out})

@app.route("/export_comp_graph_csv")
def export_comp_graph_csv():
    global COMP_GRAPH
    if not COMP_GRAPH:
        return Response("No comp graph to export",status=400)
    csv_str= graph_to_csv(COMP_GRAPH)
    return Response(csv_str,mimetype="text/csv",
                    headers={"Content-disposition":"attachment; filename=comp_graph_data.csv"})

@app.route("/export_q_table")
def export_q_table_api():
    global Q_TABLE
    data={}
    for s in Q_TABLE:
        data[str(s)] = Q_TABLE[s]
    return jsonify({"message":"Q-Table exported","table":data})

@app.route("/export_q_table_csv")
def export_q_table_csv():
    import io
    global Q_TABLE
    output= io.StringIO()
    output.write("state,a0,a1,a2,a3\n")
    for s in sorted(Q_TABLE.keys()):
        row= [str(s)] + [str(val) for val in Q_TABLE[s]]
        output.write(",".join(row)+"\n")
    csv_str= output.getvalue()
    output.close()
    return Response(csv_str,mimetype="text/csv",
                    headers={"Content-disposition":"attachment; filename=q_table.csv"})

@app.route("/export_comp_graph_layer/<layer_str>")
def export_comp_graph_layer_json(layer_str):
    global COMP_GRAPH
    if not COMP_GRAPH:
        return jsonify({"error":"No COMP_GRAPH. Run first!"})
    try:
        layer_num= int(layer_str)
    except:
        return jsonify({"error":f'Invalid layer number: {layer_str}'})
    subg= get_layer_subgraph(COMP_GRAPH, layer_num)
    if subg.number_of_nodes()==0:
        return jsonify({"error":f'No comp graph BFS_distance={layer_num}'})
    data_out= graph_to_json(subg)
    return jsonify({"message":f'CompGraph BFS layer={layer_num}',"data":data_out})

@app.route("/export_comp_graph_layer_csv/<layer_str>")
def export_comp_graph_layer_csv(layer_str):
    global COMP_GRAPH
    if not COMP_GRAPH:
        return Response("No COMP_GRAPH. Run first!",status=400)
    try:
        layer_num= int(layer_str)
    except:
        return Response(f"Invalid layer number: {layer_str}", status=400)
    subg= get_layer_subgraph(COMP_GRAPH, layer_num)
    if subg.number_of_nodes()==0:
        return Response(f"No compgraph BFS_distance={layer_num}", status=400)
    csv_str= graph_to_csv(subg)
    return Response(
        csv_str,
        mimetype="text/csv",
        headers={"Content-disposition":f"attachment; filename=compgraph_layer_{layer_num}.csv"}
    )

###############################################################################
# Color, mem/time converge
###############################################################################

def mark_graph_changes(new_graph, old_graph):
    if not old_graph:
        for n in new_graph.nodes:
            new_graph.nodes[n]["color"] = "green"
        for e in new_graph.edges:
            new_graph[e[0]][e[1]]["color"] = "green"
        return

    old_nodes= old_graph.nodes(data=True)
    old_edges= set(old_graph.edges())

    for n,d in new_graph.nodes(data=True):
        if n not in old_graph:
            d["color"]="green"
        else:
            old_data= old_nodes[n]
            changed=False
            fields= ["BFS_distance","mem","timecost","target_mem","target_time","flops"]
            for f in fields:
                old_val= old_data.get(f,None)
                new_val= d.get(f,None)
                if old_val!=new_val:
                    changed=True
                    break
            if changed:
                d["color"]="red"
            else:
                d["color"]="blue"

    new_edges= set(new_graph.edges())
    for e in new_edges:
        if e not in old_edges and (e[1],e[0]) not in old_edges:
            new_graph[e[0]][e[1]]["color"]="green"
        else:
            new_graph[e[0]][e[1]]["color"]="gray"

def converge_mem_time():
    global COMP_GRAPH
    alpha=0.3
    if COMP_GRAPH:
        for n,d in COMP_GRAPH.nodes(data=True):
            if "mem" in d and "target_mem" in d:
                old_m= d["mem"]
                tgt_m= d["target_mem"]
                d["mem"] = old_m + alpha*(tgt_m - old_m)
            if "timecost" in d and "target_time" in d:
                old_t= d["timecost"]
                tgt_t= d["target_time"]
                d["timecost"]= old_t + alpha*(tgt_t - old_t)

def background_loop():
    while True:
        time.sleep(3)
        converge_mem_time()

bg_thread= threading.Thread(target=background_loop, daemon=True)
bg_thread.start()

###############################################################################
# Node Drag => BFS => Recalc
###############################################################################

@app.route("/update_node_position", methods=["POST"])
def update_node_position():
    global COMP_GRAPH, COMP_SHAPE_INFO
    if not COMP_GRAPH:
        return jsonify({"error":"No comp graph"})

    data= request.get_json(force=True)
    node_id= data.get("node","")
    x_pos= float(data.get("x",0))
    y_pos= float(data.get("y",0))

    if node_id not in COMP_GRAPH:
        return jsonify({"error":f'Node {node_id} not in comp graph'})

    COMP_GRAPH.nodes[node_id]["x"]= x_pos
    COMP_GRAPH.nodes[node_id]["y"]= y_pos

    compute_bfs_levels(COMP_GRAPH)

    seq_len    = COMP_GRAPH.graph.get("seq_len",32)
    attn_comp  = COMP_GRAPH.graph.get("attn_comp","Quadratic (n^2)")
    lin_comp   = COMP_GRAPH.graph.get("lin_comp","Linear (n)")
    def_comp   = COMP_GRAPH.graph.get("def_comp","Linear (n)")
    scale_factor= COMP_GRAPH.graph.get("scale_factor",1.0)

    for n,d in COMP_GRAPH.nodes(data=True):
        BFS_d= d.get("BFS_distance",0)
        shape_in, shape_out= COMP_SHAPE_INFO.get(n, ("?","?"))
        if isinstance(shape_out, tuple):
            numel=1
            for dim in shape_out:
                numel*= dim
            shape_mem= (numel*4)/1024
        else:
            fallback_elems= random.randint(1,10)*1000
            shape_mem= (fallback_elems*4)/1024

        node_str= n.replace("COMP_","")
        base_t= interpret_time_cost(node_str, attn_comp, lin_comp, def_comp, seq_len)
        base_f= compute_node_flops(node_str, attn_comp, lin_comp, def_comp, seq_len)

        mem_init= (BFS_d+1)* shape_mem* scale_factor
        time_init= (BFS_d+1)* base_t
        flop_val= (BFS_d+1)* base_f

        d["mem"]= mem_init
        d["target_mem"]= mem_init
        d["timecost"]= time_init
        d["target_time"]= time_init*1.5
        d["flops"]= flop_val
        d["color"]="red"

    BFS_d= COMP_GRAPH.nodes[node_id].get("BFS_distance",0)
    return jsonify({
        "message":"Node BFS => recalc => random fallback => unique FLOPs/time",
        "node": node_id,
        "BFS_d": BFS_d,
        "x": x_pos,
        "y": y_pos
    })

###############################################################################
# poll => BFS,mem,time,flops
###############################################################################

@app.route("/poll_compgraph_data")
def poll_compgraph_data():
    global COMP_GRAPH
    if not COMP_GRAPH:
        return jsonify({})
    out={}
    for n,d in COMP_GRAPH.nodes(data=True):
        BFS_d= d.get("BFS_distance",0)
        memv= d.get("mem",0)
        timv= d.get("timecost",0)
        flp= d.get("flops",0)
        out[n]={
            "BFS_distance": BFS_d,
            "mem": round(memv,2),
            "timecost": round(timv,2),
            "flops": round(flp,2)
        }
    return jsonify(out)

###############################################################################
# MAIN PAGE
###############################################################################

@app.route("/", methods=["GET","POST"])
def single_page():
    global HYP_GRAPH, COMP_GRAPH, OLD_HYP_GRAPH, OLD_COMP_GRAPH
    global COMP_SHAPE_INFO

    subgraph_html=""
    graph_html=""

    try:
        if request.method=="POST":
            model_label= request.form.get("model","BERT (base)")
            hf_id     = AVAILABLE_MODELS.get(model_label,"bert-base-uncased")
            gpu_sel   = request.form.get("gpu","T4")
            quant     = request.form.get("quant","None (FP32)")
            reasoning = request.form.get("reason","No reasoning")
            rl_algo   = request.form.get("rl_algo","No RL")
            user_txt  = request.form.get("textinput","").strip()
            act       = request.form.get("act","run")

            layer_input= request.form.get("layer_input","").strip()
            hyper_filter= request.form.get("hyper_filter","").strip()

            seq_len_str= request.form.get("seq_len","32").strip()
            attn_comp_choice= request.form.get("attn_comp","Quadratic (n^2)")
            lin_comp_choice = request.form.get("lin_comp","Linear (n)")
            def_comp_choice = request.form.get("def_comp","Linear (n)")

            try:
                seq_len= int(seq_len_str)
                if seq_len<1: seq_len=1
            except:
                seq_len=32

            model, tok = load_model(hf_id, quant)

            if rl_algo!="No RL":
                s= random.randint(0,9)
                a= random.randint(0,3)
                r= random.uniform(-1,1)
                update_q_table(s,a,r)

            if act in ["run","fine_tune","speak"]:
                if act=="fine_tune":
                    toy_finetune(model)

                new_hyp= extract_hypergraph(model, quant, reasoning)
                new_comp= extract_comp_graph(model, tok, gpu_sel, quant, reasoning)
                compute_bfs_levels(new_comp)
                mark_graph_changes(new_hyp, OLD_HYP_GRAPH)
                mark_graph_changes(new_comp, OLD_COMP_GRAPH)
                HYP_GRAPH= new_hyp
                COMP_GRAPH= new_comp
                OLD_HYP_GRAPH= new_hyp.copy()
                OLD_COMP_GRAPH= new_comp.copy()

                shape_dict= trace_model_fx(model,tok)
                COMP_SHAPE_INFO.clear()
                for node_name,(inp,outp) in shape_dict.items():
                    nm= "COMP_"+ node_name
                    COMP_SHAPE_INFO[nm]= (inp,outp)

                scale_factor= GPU_MEMORY_SCALE.get(gpu_sel,1.0)
                COMP_GRAPH.graph["seq_len"]= seq_len
                COMP_GRAPH.graph["attn_comp"]= attn_comp_choice
                COMP_GRAPH.graph["lin_comp"]= lin_comp_choice
                COMP_GRAPH.graph["def_comp"]= def_comp_choice
                COMP_GRAPH.graph["scale_factor"]= scale_factor

                # BFS => fill memory/time/flops
                for n,d2 in COMP_GRAPH.nodes(data=True):
                    BFS_d= d2.get("BFS_distance",0)
                    node_str= n.replace("COMP_","")

                    shape_in, shape_out= shape_dict.get(node_str, ("?","?"))
                    if isinstance(shape_out, tuple):
                        numel=1
                        for dim in shape_out:
                            numel*= dim
                        shape_mem= (numel*4)/1024
                    else:
                        fallback_elems= random.randint(1,10)*1000
                        shape_mem= (fallback_elems*4)/1024

                    base_t= interpret_time_cost(node_str, attn_comp_choice, lin_comp_choice, def_comp_choice, seq_len)
                    base_f= compute_node_flops(node_str, attn_comp_choice, lin_comp_choice, def_comp_choice, seq_len)

                    mem_init= (BFS_d+1)* shape_mem* scale_factor
                    time_init= (BFS_d+1)* base_t
                    flop_val= (BFS_d+1)* base_f

                    d2["mem"]= mem_init
                    d2["target_mem"]= mem_init
                    d2["timecost"]= time_init
                    d2["target_time"]= time_init*1.5
                    d2["flops"]= flop_val

            elif act=="layer_view":
                if COMP_GRAPH:
                    try:
                        ln= int(layer_input)
                        subg= get_layer_subgraph(COMP_GRAPH, ln)
                        if subg.number_of_nodes()==0:
                            subgraph_html= f"<h4>No BFS={ln}</h4>"
                        else:
                            from pyvis.network import Network
                            net_c= Network(height='500px',width='600px',directed=False)
                            net_c.repulsion(node_distance=140,central_gravity=0.1)
                            for nd,dd in subg.nodes(data=True):
                                BFS_d= dd.get("BFS_distance",0)
                                memv= dd.get("mem",0)
                                timv= dd.get("timecost",0)
                                flps= dd.get("flops",0)
                                colr= dd.get("color","blue")
                                txt= f"BFS={BFS_d}, mem={memv:.2f} KB, time={timv:.2f} ms, flops={flps}"
                                net_c.add_node(nd,label=nd,shape="dot",size=20,borderWidth=1,
                                               title=txt,color={"border":"black","background":colr})
                            for (uI,vI) in subg.edges():
                                cI= subg[uI][vI].get("color","gray")
                                net_c.add_edge(uI,vI,color=cI)
                            net_c.write_html("static/layer_view.html", notebook=False, open_browser=False)
                            subgraph_html= f"<h3>Layer BFS={ln}</h3><iframe src='/static/layer_view.html' width='600' height='500' style='border:none;overflow:hidden;' scrolling='no'></iframe>"
                    except:
                        subgraph_html= f"<h4>Invalid BFS layer: {layer_input}</h4>"
                else:
                    subgraph_html= "<h4>No compgraph, run first</h4>"

            elif act=="hyper_filter_view":
                if HYP_GRAPH:
                    from pyvis.network import Network
                    fl= hyper_filter.lower()
                    subg= nx.Graph()
                    for nodeH,dataH in HYP_GRAPH.nodes(data=True):
                        if fl in nodeH.lower():
                            subg.add_node(nodeH,**dataH)
                    for (uH,vH) in HYP_GRAPH.edges():
                        if (uH in subg.nodes) and (vH in subg.nodes):
                            subg.add_edge(uH,vH,**HYP_GRAPH[uH][vH])
                    if subg.number_of_nodes()==0:
                        subgraph_html= f"<h4>No hypergraph match '{hyper_filter}'</h4>"
                    else:
                        net_h= Network(height='500px',width='600px',directed=False)
                        net_h.repulsion(node_distance=140,central_gravity=0.1)
                        for nH,dH in subg.nodes(data=True):
                            net_h.add_node(nH,label=nH,shape="dot",size=20,borderWidth=1,
                                           title=nH,color={"border":"black","background":"blue"})
                        for (uZ,vZ) in subg.edges():
                            net_h.add_edge(uZ,vZ,color="gray")
                        net_h.write_html("static/hyper_filter_view.html",notebook=False,open_browser=False)
                        subgraph_html= f"<h3>Hypergraph Filter='{hyper_filter}'</h3><iframe src='/static/hyper_filter_view.html' width='600' height='500' style='border:none;overflow:hidden;' scrolling='no'></iframe>"
                else:
                    subgraph_html= "<h4>No hypergraph found, run first!</h4>"

        # if we have graphs => build final
        if HYP_GRAPH and COMP_GRAPH:
            from pyvis.network import Network
            if "BFS_distance" not in list(COMP_GRAPH.nodes(data=True))[0][1]:
                compute_bfs_levels(COMP_GRAPH)

            net_h= Network(height='500px',width='600px',directed=False)
            net_h.repulsion(node_distance=140,central_gravity=0.1)
            for nd,dd in HYP_GRAPH.nodes(data=True):
                c8= dd.get("color","blue")
                net_h.add_node(nd,label=nd,shape="dot",size=20,borderWidth=1,
                               title=nd,color={"border":"black","background":c8})
            for s8,t8 in HYP_GRAPH.edges():
                ec8= HYP_GRAPH[s8][t8].get("color","gray")
                net_h.add_edge(s8,t8,color=ec8)
            net_h.write_html("static/hypergraph.html", notebook=False, open_browser=False)

            net_c= Network(height='500px',width='600px',directed=False)
            net_c.repulsion(node_distance=140,central_gravity=0.1)
            for nd2,dd2 in COMP_GRAPH.nodes(data=True):
                BFS_d= dd2.get("BFS_distance",0)
                m2= dd2.get("mem",0)
                t2= dd2.get("timecost",0)
                f2= dd2.get("flops",0)
                col2= dd2.get("color","blue")
                txt2= f"BFS={BFS_d}, mem={m2:.2f} KB, time={t2:.2f} ms, flops={f2}"
                net_c.add_node(nd2,label=nd2,shape="dot",size=20,borderWidth=1,
                               title=txt2,color={"border":"black","background":col2})
            for (ua,va) in COMP_GRAPH.edges():
                ccl= COMP_GRAPH[ua][va].get("color","gray")
                net_c.add_edge(ua,va,color=ccl)
            net_c.write_html("static/compgraph.html", notebook=False, open_browser=False)

            with open("static/compgraph.html","r") as f:
                c_html= f.read()

            drag_js= """
<script>
  var throttleTimeout=null;
  network.on("drag",(params)=>{
    if(params.nodes.length>0){
      if(throttleTimeout)return;
      throttleTimeout=setTimeout(()=>{
        throttleTimeout=null;
        let nodeId=params.nodes[0];
        let nodePos=network.getPosition(nodeId);
        fetch("/update_node_position",{
          method:"POST",
          headers:{"Content-Type":"application/json"},
          body:JSON.stringify({"node":nodeId,"x":nodePos.x,"y":nodePos.y})
        })
        .then(rr=>rr.json())
        .then(dat=>{
          console.log("Drag BFS =>",dat);
          fetch("/poll_compgraph_data")
          .then(r2=>r2.json())
          .then(d2=>{
            for(let keyx in d2){
              let BFS_d=d2[keyx].BFS_distance;
              let memx=d2[keyx].mem.toFixed(2);
              let timx=d2[keyx].timecost.toFixed(2);
              let flpx=d2[keyx].flops.toFixed(2);
              if(network.body.nodes[keyx]){
                network.body.nodes[keyx].options.title=
                  "BFS="+BFS_d+", mem="+memx+" KB, time="+timx+" ms, flops="+flpx;
              }
            }
            network.redraw();
          })
          .catch(e2=>console.error(e2));
        })
        .catch(err=>console.error(err));
      },150);
    }
  });
</script>
"""

            poll_js= """
<script>
setInterval(()=>{
  fetch("/poll_compgraph_data")
  .then(r=>r.json())
  .then(dat=>{
    for(let nodeId in dat){
      let BFS_d=dat[nodeId].BFS_distance;
      let memv=dat[nodeId].mem.toFixed(2);
      let timv=dat[nodeId].timecost.toFixed(2);
      let flp =dat[nodeId].flops.toFixed(2);
      if(network.body.nodes[nodeId]){
        network.body.nodes[nodeId].options.title=
          "BFS="+BFS_d+", mem="+memv+" KB, time="+timv+" ms, flops="+flp;
      }
    }
    network.redraw();
  })
  .catch(er=>console.error(er));
},3000);
</script>
"""

            c_html= c_html.replace("</body>", drag_js+"\n"+poll_js+"\n</body>")
            with open("static/compgraph.html","w") as f:
                f.write(c_html)

            graph_html= f"""
<div style="margin:auto;width:90%;text-align:center;">
  <table style="margin:auto;"><tr>
     <td valign="top">
       <h2>Hypergraph</h2>
       <iframe src="/static/hypergraph.html" width="600" height="500"
         style="border:none;overflow:hidden;" scrolling="no"></iframe>
       <p><a href="/glossary_hyper" target="_blank">Hypergraph Glossary</a></p>
     </td>
     <td valign="top">
       <h2>Computational Graph</h2>
       <iframe src="/static/compgraph.html" width="600" height="500"
         style="border:none;overflow:hidden;" scrolling="no"></iframe>
       <p><a href="/glossary_comp" target="_blank">CompGraph Glossary</a></p>
     </td>
  </tr></table>
</div>

<div style="margin:auto;width:90%;text-align:center;">
  <h2>3D Q-Space</h2>
  <iframe src="/q_space" width="700" height="600"
    style="border:none;overflow:hidden;" scrolling="no"></iframe>
  <p>Drag to orbit the Q-Space!</p>
</div>

<div style="margin:auto;width:90%;text-align:center;">
  <h3>Data Exports</h3>
  <p>
    <a href="/export_hypergraph" target="_blank">Hypergraph JSON</a> |
    <a href="/export_hypergraph_csv" target="_blank">Hypergraph CSV</a><br>
    <a href="/export_comp_graph" target="_blank">CompGraph JSON</a> |
    <a href="/export_comp_graph_csv" target="_blank">CompGraph CSV</a><br>
    <a href="/export_q_table" target="_blank">Q-Table JSON</a> |
    <a href="/export_q_table_csv" target="_blank">Q-Table CSV</a>
  </p>
  <p>
    <a href="/analysis_3d_scatter" target="_blank">3D Scatter: Mem vs. Time vs. FLOPs (color=BFS)</a>
  </p>
</div>
"""

        style_center= "style='text-align:center; margin:auto; width:90%;'"
        model_opts= "".join([f'<option value="{lbl}">{lbl}</option>' for lbl in AVAILABLE_MODELS])
        gpu_opts= "".join([f'<option value="{gp}">{gp}</option>' for gp in AVAILABLE_GPUS])
        quant_opts= "".join([f'<option value="{qq}">{qq}</option>' for qq in QUANT_METHODS])
        reas_opts= "".join([f'<option value="{rr}">{rr}</option>' for rr in REASONING_METHODS])
        rl_opts= "".join([f'<option value="{alg}">{alg}</option>' for alg in RL_ALGORITHMS])
        comp_opts= "".join([f'<option value="{co}">{co}</option>' for co in COMPLEXITY_OPTS])

        html= f"<html><head><title>AI Interpretability</title></head><body {style_center}>"
        html+= "<h1>AI Interpretability</h1>"
        html+= "<p><i>Memory in KB, Time in ms, FLOPs unitless. BFS-based node distance. No conversation logs!</i></p>"

        # The main form
        html+= "<form method='post'>"
        html+= "<label>Model:</label>"
        html+= f"<select name='model'>{model_opts}</select>"
        html+= "<label> GPU:</label>"
        html+= f"<select name='gpu'>{gpu_opts}</select>"
        html+= "<label> Quant:</label>"
        html+= f"<select name='quant'>{quant_opts}</select>"
        html+= "<label> Reasoning:</label>"
        html+= f"<select name='reason'>{reas_opts}</select>"
        html+= "<label> RL:</label>"
        html+= f"<select name='rl_algo'>{rl_opts}</select>"
        html+= "<br><br>"

        html+= "<label>Seq Len (n):</label>"
        html+= '<input name="seq_len" value="32" style="width:80px;" />'

        html+= "<br><label>Attention Complexity:</label>"
        html+= f"<select name='attn_comp'>{comp_opts}</select>"
        html+= "<label> Linear:</label>"
        html+= f"<select name='lin_comp'>{comp_opts}</select>"
        html+= "<label> Default:</label>"
        html+= f"<select name='def_comp'>{comp_opts}</select>"

        html+= "<br><br><textarea name='textinput' rows='3' cols='60' placeholder='Optional user text...'></textarea><br><br>"

        html+= "<button name='act' value='run'>Run</button> "
        html+= "<button name='act' value='fine_tune'>Fine-Tune</button> "
        html+= "<button name='act' value='speak'>Speak</button> "

        html+= "<hr>"
        html+= '<label>BFS Layer #:</label>'
        html+= '<input name="layer_input" placeholder="1" /> '
        html+= '<button name="act" value="layer_view">Show BFS Layer</button>'

        html+= "<hr>"
        html+= '<label>Hypergraph Filter:</label>'
        html+= '<input name="hyper_filter" placeholder="attention or layer.1" /> '
        html+= '<button name="act" value="hyper_filter_view">Filter Hypergraph</button>'

        html+= "</form><br>"

        html+= subgraph_html
        html+= graph_html

        html+= "</body></html>"
        return html

    except Exception as ex:
        tb= traceback.format_exc()
        return f"<h3>Internal Error</h3><pre>{tb}</pre>", 500

if __name__=="__main__":
    print("Starting Flask on port=8000, debug=False ...")
    try:
        app.run(host="0.0.0.0", port=8000, debug=False)
    except KeyboardInterrupt:
        print("KeyboardInterrupt => Exiting gracefully.")
        pass
'''

# Write interpret_flask.py
with open("interpret_flask.py","w") as f:
    f.write(FULL_CODE)

print("✅ Wrote interpret_flask.py (3D Q-Space orbit, 3D scatter route, no logs). Starting flask in background...")

# Start Flask in background
process = Popen(["python", "interpret_flask.py"], shell=False)

print("⏳ Polling localhost:8000 for up to 30s to confirm server is up...")

ready=False
for i in range(30):
    time.sleep(1)
    try:
        r = requests.get("http://127.0.0.1:8000")
        if r.status_code==200:
            print(f"✅ Flask responded after {i+1} sec. OK!")
            ready=True
            break
    except:
        pass

if not ready:
    print("⚠️ Could not confirm Flask after 30s. Attempting ngrok anyway...")

print("🔗 Starting ngrok =>")
from pyngrok import ngrok

NGROK_AUTH_TOKEN="2tedizGJDmEJMDtfhCR6ftx8EyX_2RaA7t8nPn17SU7jiyK1H"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url= ngrok.connect(8000)
print("🌐 Public URL =>", public_url)

print("✅ Done. The Q-Space is orbitable, plus /analysis_3d_scatter for memory-time-FLOPs BFS color. Enjoy!")
time.sleep(2)
!tail -n 20 server.log


🔴 Killing anything on port=8000 + old ngrok sessions.
📦 Installing dependencies (quietly)...
✅ Wrote interpret_flask.py (3D Q-Space orbit, 3D scatter route, no logs). Starting flask in background...
⏳ Polling localhost:8000 for up to 30s to confirm server is up...
✅ Flask responded after 11 sec. OK!
🔗 Starting ngrok =>
🌐 Public URL => NgrokTunnel: "https://0525-35-229-205-40.ngrok-free.app" -> "http://localhost:8000"
✅ Done. The Q-Space is orbitable, plus /analysis_3d_scatter for memory-time-FLOPs BFS color. Enjoy!
tail: cannot open 'server.log' for reading: No such file or directory
